In [53]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import cPickle as pk
import pandas as pd

# Munging US CPC to IPC into data Checking Compatibility

## SQL Query used on PATSTAT 2015 Spring

## Importing
[to top](#Table-of-Contents) 

In [54]:
pwd

u'/Users/innovation-sfi/Dropbox/Research/Innovation/EPO/Data_processing/Python'

# CPC

In [55]:
f_all_grants = '../Data_Play/all_grant_only_prior_US_cpc_v1.csv'

In [56]:
USdf = pd.read_csv(f_all_grants, engine='c', dtype={'publn_nr':str,'appln_id':str})
#dont need ID's

In [57]:
print USdf.head()
print USdf.shape
print USdf.describe()

  publn_nr appln_auth  publn_date  appln_id cpc_class_symbol
0   718793         US  1903-01-20  46789746      F24C   1/02
1  1113768         US  1914-10-13  47666487    A61M   1/0005
2  1113768         US  1914-10-13  47666487     F04B  53/164
3  1216480         US  1917-02-20  47845187      H04M   1/02
4   353089         US  1886-11-23  46162467     A63B  21/062
(15150421, 5)
        publn_nr appln_auth  publn_date  appln_id cpc_class_symbol
count   15150421   15150421    15150421  15150421         15150421
unique   5578666          1        9638   5583429           217189
top      6861964         US  2014-09-16  53279350    H01L2924/0002
freq         337   15150421       11419       337            17047


Most common code is Indexing scheme for arrangements or methods for connecting or disconnecting semiconductor or solid-state bodies as covered by H01L 24/00

NOTE: Codes H01L 2924/0001 to H01L 2924/0002 are used to describe the status of reclassification; they do not relate to technical features as such

. H01L 2924/0001 Technical content checked by a classifier

. . H01L 2924/0002 Not covered by any one of groups H01L 24/00, H01L 24/00 and H01L 2224/00

In [58]:
s_US_cd_len = USdf.cpc_class_symbol
s_US_cd_len.map(lambda x: x.split('/')[0])

0           F24C   1
1           A61M   1
2           F04B  53
3           H04M   1
4           A63B  21
5           A63B  21
6           B27M   3
7           E21B  17
8           E21B  43
9           B63C   7
10          D01H   7
11          D01H   7
12          D07B   3
13          A61F   2
14          A61F   2
15          A61F   2
16          Y10T  29
17          Y10T  29
18          Y10T  29
19          B68B   3
20          B68B   3
21          B22C   9
22          B41B   9
23          C08K   3
24          C08K   5
25          C09D   7
26          C09D   7
27          C09D 189
28          D06M  15
29          A01B   1
              ...   
15150391    A61H  15
15150392    A61L   9
15150393    A45D   8
15150394    A61H  19
15150395    A45D  24
15150396    A45D  44
15150397    A45D  44
15150398    A61H  19
15150399    A45D   8
15150400    A61H  15
15150401    A61H  23
15150402    A45D  24
15150403    G06F   3
15150404    G06F   3
15150405    G06F   3
15150406    G06F   3
15150407    G

In [59]:
# showing that even in CPC, spliting on '/' leaves codes of all same length -  good regarding matching
USdf.cpc_class_symbol.map(lambda x: x.split('/')[0]).map(len).describe()

count    15150421
mean            8
std             0
min             8
25%             8
50%             8
75%             8
max             8
Name: cpc_class_symbol, dtype: float64

In [75]:
# showing using lack of white space ' ' can filter out CPC only codes
s_cpc_only = USdf.cpc_class_symbol[~USdf.cpc_class_symbol.str.contains(' ')]; s_cpc_only

50           A61F2002/7862
57           A61F2002/7862
74           A61F2002/5073
84             C04B2235/32
85           C04B2235/3409
86           C04B2235/3817
87           C04B2235/3821
88            C04B2235/421
89            C04B2235/424
90            C04B2235/425
91            C04B2235/528
92           C04B2235/5427
96             A41D2400/38
107          A61F2002/5072
133          A61F2002/5072
145          A61F2002/5072
155          A61F2002/5072
164          A61F2002/6621
171           F17C2201/052
192          A61F2002/5072
199            E01D2101/22
200            E01D2101/24
201           E01D2101/268
210          A61F2002/5009
211           A61F2002/502
253           B26F2001/365
257          A61F2002/5072
271          A61F2002/5072
273          A61F2002/5007
285          F16H2007/0806
                 ...      
15150171      F21Y2111/007
15150189      A45C2011/002
15150204      F23J2213/202
15150205       F23J2213/50
15150224       F21Y2101/02
15150225      F21Y2105/001
1

In [79]:
# fraction of codes that are CPC only in US CPC
2427238/float(15150421)

0.16020927735275475

In [81]:
# WRONG, this using CPC codes for ALL US, but we have IPC for most
# fraction of codes that are CPC only in all set 
2427238/float(15150421+45538451)

0.03999477861443857

In [82]:
# WRONG, this is CPC codes for ALL US, but we have IPC for most
#fraction of codes that will come just from US CPC
15150421/float(15150421+45538451)
# THAT"S ALOT

0.24964084025157035

### Want to check if can replace middle of these 'CPC only' codes with white space and get cpc codes

Think this should be possible as they all follow this format:
A01D  23/00
A01D  23/02 
A01D2023/022
A01D2023/024 
A01D2023/026 
A01D2023/028 
A01D  23/04
A01D  23/06

But examples like
A01D2000/028 are counter exampls, there isn't any groups that are 00 i.e not exist A01D  00/028

In [62]:
tot_file = '../Data_Play/all_grant_only_prior_v1.csv'

In [63]:
df = pd.read_csv(tot_file, engine='c', dtype={'publn_nr':str,'appln_id':str})
#dont need ID's

In [64]:
print df.head()
df.shape

  publn_nr appln_auth  publn_date  appln_id ipc_class_symbol
0  PP19626         US  2009-01-20  55754593      A01H   5/00
1  PP19627         US  2009-01-20  52899962      A01H   5/00
2  PP19628         US  2009-01-20  55754594      A01H   5/00
3  PP19629         US  2009-01-20  55754595      A01H   5/00
4  PP19630         US  2009-01-20  55318791      A01H   5/00


(45538451, 5)

In [74]:
df[~df.ipc_class_symbol.str.contains(' ')].head()
# seems to imply no A01B1001/00 codes in IPC
# further check below

,publn_nr,appln_auth,publn_date,appln_id,ipc_class_symbol
645,7479062,US,2009-01-20,46780856,G06F
779,7479168,US,2009-01-20,49994785,C07C
782,7479168,US,2009-01-20,49994785,C10G
1497,7479686,US,2009-01-20,49344017,H01L
1941,7480056,US,2009-01-20,53869339,G02F


In [70]:
# fraction of codes that are only at subclass level
429396/float(45538451)
# good

0.009429306236173909

In [71]:
df.ipc_class_symbol[~df.ipc_class_symbol.str.contains(' ')].map(len).value_counts()
# good

4    429396
Name: ipc_class_symbol, dtype: int64